# Imports

In [1]:
import pandas as pd
import numpy as np
from requests import get
from bs4 import BeautifulSoup

from time import sleep

from random import randint

from time import time

from warnings import warn 
warn("Warning Simulation")

from IPython.core.display import clear_output

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Warning Simulation
  del sys.path[0]


In [2]:
url = 'https://www.imdb.com/search/title/?release_date=2019&sort=num_votes,desc&page=1'
response = get(url)
print(response.text[:500])




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle"


In [3]:
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

**Store all of our html in an object**

In [4]:
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


## Movie Titles

In [5]:
first_movie = html_soup.find('div', class_ = 'lister-item mode-advanced')

In [6]:
first_movie.div

<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt7286456"></div>
</div>

In [7]:
first_movie.a

<a href="/title/tt7286456/"> <img alt="Joker" class="loadlate" data-tconst="tt7286456" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BNGVjNWI4ZGUtNzE0MS00YTJmLWE0ZDctN2ZiYTk2YmI3NTYyXkEyXkFqcGdeQXVyMTkxNjUyNQ@@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB466725069_.png" width="67"/>
</a>

In [8]:
first_movie.h3

<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt7286456/">Joker</a>
<span class="lister-item-year text-muted unbold">(2019)</span>
</h3>

In [9]:
first_movie.h3.a

<a href="/title/tt7286456/">Joker</a>

In [10]:
first_name = first_movie.h3.a.text
first_name

'Joker'

## Movie Release Year

In [11]:
first_year = first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold')
first_year

<span class="lister-item-year text-muted unbold">(2019)</span>

In [12]:
first_year = first_year.text
first_year

'(2019)'

In [13]:
first_movie.strong

<strong>8.5</strong>

## IMDB score

In [14]:
first_imdb = float(first_movie.strong.text)
first_imdb

8.5

## Metascore

In [15]:
first_mscore = first_movie.find('span', class_ = 'metascore mixed')
first_mscore = int(first_mscore.text)
first_mscore

59

## Number of votes on IMDB

In [16]:
first_votes = first_movie.find('span', attrs = {'name' : 'nv'})
first_votes

<span data-value="836141" name="nv">836,141</span>

In [17]:
first_votes['data-value']

'836141'

In [18]:
first_votes = int(first_votes['data-value'])
first_votes

836141

## For-loop that populates a list with movies from IMDB
- after our for-loop we convert our list into a dataframe 

In [19]:
# Lists to store the scraped data
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
# Extract data from individual movie container
for contain in movie_containers:
# If the movie has a metascore, extract it
    if contain.find('div', class_ = 'ratings-metascore') is not None:
# The movie title
        name = contain.h3.a.text
        names.append(name)
# Year the movie was released
        year = contain.h3.find('span', class_ = 'lister-item-year').text
        years.append(year)
# The IMDB movie rating
        imdb = float(contain.strong.text)
        imdb_ratings.append(imdb)
# The metascore
        meta = contain.find('span', class_ = 'metascore').text
        metascores.append(int(meta))
# The number of votes on imdb
        vote = contain.find('span', attrs = {'name' : 'nv'})['data-value']
        votes.append(int(vote))

In [20]:
# testing the results of converting our list to a dataframe
test_df = pd.DataFrame({'movie' : names,
                       'year' : year,
                       'imdb' : imdb_ratings,
                       'metascore' : metascores,
                       'votes':votes})
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movie      39 non-null     object 
 1   year       39 non-null     object 
 2   imdb       39 non-null     float64
 3   metascore  39 non-null     int64  
 4   votes      39 non-null     int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.6+ KB


## Scripting multiple pages

In [21]:
# this is a test to see how long it will take to run 4 pages
pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(2000,2020)]

In [22]:
start_time = time()
requests = 0
for _ in range(5):
# A request would go here
    requests += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait=True)

Request: 5; Frequency: 0.33290482153634265 requests/s


## Final Product

In [23]:
# empty lists that we will convert to a df @ the end
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# setting up the url's we want to scrape
pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(2000,2020)]


# For every year in the interval 2000-2020
for year_url in years_url:
    
    # For every page in the interval 1-4
    for page in pages:
        
        # make a get request
        response = get('https://www.imdb.com/search/title/?release_date=' + year_url
                      + '&sort=num_votes,desc&page=' + page)
        
        # Pause the loop
        sleep(randint(8,15))
        
        # Monitor the requests
        requests += 1
        sleep(randint(1,3))
        current_time = time()
        elapsed_time = current_time - start_time
        print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)
        
        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn("Number of requests was greater than expected.")
            break
            
        # Parse the content of the request with Beautiful Soup
        page_html = BeautifulSoup(response.text, 'html.parser')
        
        # Select all 50 movie containers from a single parse
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
        
        # For every movie of these 50
        for container in mv_containers:
            # If movie has a metascore, then
            if container.find('div', class_ = 'ratings-metascore') is not None:
                
                # Scrape the name
                name = container.h3.a.text
                names.append(name)
                
                # Scrape the year
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)
                
                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)
                
                # Scrape the metascore
                meta = container.find('span', class_ = 'metascore').text
                metascores.append(int(meta))
                
                # Scrape the number of votes from IMDB
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

Request: 80; Frequency: 0.06775818897680343 requests/s


In [24]:
movies = pd.DataFrame({
    'movie':names,
    'year':years,
    'imdb_rating':imdb_ratings,
    'metascore':metascores,
    'votes':votes
})
print(movies.info())
movies.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3616 entries, 0 to 3615
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie        3616 non-null   object 
 1   year         3616 non-null   object 
 2   imdb_rating  3616 non-null   float64
 3   metascore    3616 non-null   int64  
 4   votes        3616 non-null   int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 141.4+ KB
None


,movie,year,imdb_rating,metascore,votes
0,Gladiator,(2000),8.5,67,1302823
1,Memento,(2000),8.4,80,1093977
2,Snatch,(2000),8.3,55,763994
3,Requiem for a Dream,(2000),8.3,68,745673
4,X-Men,(2000),7.4,64,560533
5,Cast Away,(2000),7.8,73,508618
6,American Psycho,(2000),7.6,64,470049
7,Unbreakable,(2000),7.3,62,379001
8,Mission: Impossible II,(2000),6.1,59,307484
9,Meet the Parents,(2000),7.0,73,305627


In [25]:
movies.loc[:, 'year'] = movies['year'].str[-5:-1].astype(int)

In [26]:
movies['year'].head()

0    2000
1    2000
2    2000
3    2000
4    2000
Name: year, dtype: int64

In [27]:
# importing to csv
movies.to_csv('movie_ratings.csv')